In [77]:
# Import dependencies
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [78]:
# Import pre-computed BERT CLS vectors and Feature Vectors
with open('cls_emb.pkl', 'rb') as f:
    cls = pickle.load(f)
with open('feature_vectors.pkl', 'rb')as f:
    feature_vectors= pickle.load(f)

In [79]:
# The model needs numerical classes as base truth targets. Convert model names to 0, 1, 2
response_df = pd.read_csv('final_data.csv')
map_dict = {'llama3.1-70b':0, 'mistral':1, 'gpt-4o-2024-05-13':2}
response_df['model_nums'] = response_df['model'].map(map_dict)
model_nums = response_df['model_nums']

In [80]:
# Concatenate the CLS and feature vectors
embeddings = [torch.cat((cls[i].float(), torch.from_numpy(feature_vectors[i]).unsqueeze(0).float()), dim=1) for i in range(len(cls))]

# Calculate the number of features to be used later as input to model
NUM_FEATURES = embeddings[0].size(1)

In [81]:
# Create a development and validation set by splitting indices
RANDOM_STATE = 42
indices = [i for i in range(len(embeddings))]
dev_indices, val_indices = train_test_split(indices, test_size = 0.1, random_state = RANDOM_STATE)

In [95]:
def extract_and_split_dev(temp, dev=True):
    """
    Returns split train, test, and dev sets based on temperature

    temp: float indicating temperature 
    dev: boolean indicating if the return set is the dev set or validation
    """
    if dev:
        temp_embs = [embeddings[idx] for idx in dev_indices if response_df['temperature'][idx]==temp]
        temp_targs = [model_nums[idx] for idx in dev_indices if response_df['temperature'][idx]==temp]
        return train_test_split(temp_embs, temp_targs, test_size=0.2, random_state=RANDOM_STATE)
    if not dev:
        return ([embeddings[idx] for idx in val_indices if response_df['temperature'][idx]==temp], 
            [model_nums[idx] for idx in val_indices if response_df['temperature'][idx]==temp])

temp_0_train, temp_0_test, temp_0_targs_train, temp_0_targs_test = extract_and_split_dev(0)
temp_7_train, temp_7_test, temp_7_targs_train, temp_7_targs_test = extract_and_split_dev(0.7)
temp_14_train, temp_14_test, temp_14_targs_train, temp_14_targs_test = extract_and_split_dev(1.4)
temp_0_val, temp_0_val_targs = extract_and_split_dev(0, False)
temp_7_val, temp_7_val_targs = extract_and_split_dev(0.7, False)
temp_14_val, temp_14_val_targs = extract_and_split_dev(1.4, False)
temp_all_val, temp_all_val_targs = [embeddings[idx] for idx in val_indices],[model_nums[idx] for idx in val_indices]

In [85]:
temp_all_embs = [embeddings[idx] for idx in dev_indices]
temp_all_targs = [model_nums[idx] for idx in dev_indices]
temp_all_train, temp_all_test, temp_all_targs_train, temp_all_targs_test = train_test_split(
    temp_all_embs, temp_all_targs, test_size = 0.2, random_state = RANDOM_STATE)

In [98]:
with open('temp_0_val.pkl', 'wb') as f:
    pickle.dump(temp_0_val, f)
with open('temp_0_val_targs.pkl', 'wb') as f:
    pickle.dump(temp_0_val_targs, f)

with open('temp_7_val.pkl', 'wb') as f:
    pickle.dump(temp_7_val, f)
with open('temp_7_val_targs.pkl', 'wb') as f:
    pickle.dump(temp_7_val_targs, f)

with open('temp_14_val.pkl', 'wb') as f:
    pickle.dump(temp_14_val, f)
with open('temp_14_val_targs.pkl', 'wb') as f:
    pickle.dump(temp_14_val_targs, f)

with open('temp_all_val.pkl', 'wb') as f:
    pickle.dump(temp_all_val, f)
with open('temp_all_val_targs.pkl', 'wb') as f:
    pickle.dump(temp_all_val_targs, f)

In [86]:
class FAM(nn.Module):
    def __init__(self, embed_size, hidden_size, hidden_dropout_prob):
        super().__init__()
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.fc = nn.Linear(embed_size, hidden_size)
        
    def init_weights(self):
        initrange = 0.2
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()


    def forward(self, text):
        batch,  dim = text.size()
        feat = self.fc(torch.tanh(self.dropout(text.view(batch, dim))))
        feat = F.normalize(feat, dim=1)
        return feat

In [87]:
class Projection(nn.Module):
    def __init__(self, hidden_size, projection_size):
        super().__init__()
        self.fc = nn.Linear(hidden_size, projection_size)
        self.ln = nn.LayerNorm(projection_size)
        self.bn = nn.BatchNorm1d(projection_size)
        self.init_weights()
    def init_weights(self):
        initrange = 0.01
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()


    def forward(self, text):
        batch,  dim = text.size()
        return self.ln(self.fc(torch.tanh(text.view(batch, dim))))

In [88]:
class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07):
        """
        Implementation of the loss described in the paper Supervised Contrastive Learning :
        https://arxiv.org/abs/2004.11362

        :param temperature: int
        """
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, projections, targets):
        """

        :param projections: torch.Tensor, shape [batch_size, projection_dim]
        :param targets: torch.Tensor, shape [batch_size]
        :return: torch.Tensor, scalar
        """
        device = torch.device("cuda") if projections.is_cuda else torch.device("cpu")

        dot_product_tempered = torch.mm(projections, projections.T) / self.temperature
        # Minus max for numerical stability with exponential. Same done in cross entropy. Epsilon added to avoid log(0)
        exp_dot_tempered = (
            torch.exp(dot_product_tempered - torch.max(dot_product_tempered, dim=1, keepdim=True)[0]) + 1e-5
        )

        mask_similar_class = (targets.unsqueeze(1).repeat(1, targets.shape[0]) == targets).to(device)
        mask_anchor_out = (1 - torch.eye(exp_dot_tempered.shape[0])).to(device)
        mask_combined = mask_similar_class * mask_anchor_out
        cardinality_per_samples = torch.sum(mask_combined, dim=1)

        log_prob = -torch.log(exp_dot_tempered / (torch.sum(exp_dot_tempered * mask_anchor_out, dim=1, keepdim=True)))
        supervised_contrastive_loss_per_sample = torch.sum(log_prob * mask_combined, dim=1) / cardinality_per_samples
        supervised_contrastive_loss = torch.mean(supervised_contrastive_loss_per_sample)

        return supervised_contrastive_loss

In [89]:
class Classifier(nn.Module):
    def __init__(self, hidden_size, num_class, hidden_dropout_prob):
        super().__init__()
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.fc = nn.Linear(hidden_size, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.02
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, feature):
        return self.fc(torch.tanh(feature))

In [90]:
class WordEmbeddingDataset(Dataset):
    """
    Custom dataset object to feed into the dataloader
    """
    def __init__(self, embs, targs):
        """
        Class initializer

        embs: concatenated CLS embeddings and feature vectors
        targs: base truth model numbers
        """
        self.embs = embs
        self.targs = targs 

    def __len__(self):
        return len(self.embs)

    def __getitem__(self, idx):
        """
        Method used by data loader to output data
        
        idx: random batch index from dataloader

        Returns: 
        Tuple with batch embeddings at [0] and batch base truth targets at [1]
        """
        return self.embs[idx], self.targs[idx]

In [93]:
# Use batch size of 100
BATCH_SIZE = 100

# Create datasets to be used in data loaders
dataset_0 = WordEmbeddingDataset(temp_0_train, temp_0_targs_train)
dataset_0_test = WordEmbeddingDataset(temp_0_test, temp_0_targs_test)

dataset_7 =  WordEmbeddingDataset(temp_7_train, temp_7_targs_train)
dataset_7_test = WordEmbeddingDataset(temp_7_test, temp_7_targs_test)

dataset_14 = WordEmbeddingDataset(temp_14_train, temp_14_targs_train)
dataset_14_test = WordEmbeddingDataset(temp_14_test, temp_14_targs_test)

dataset_all = WordEmbeddingDataset(temp_all_train, temp_all_targs_train)
dataset_all_test = WordEmbeddingDataset(temp_all_test, temp_all_targs_test)

# Create all dataloaders
data_loader_0_train = DataLoader(dataset_0, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
data_loader_0_test = DataLoader(dataset_0_test, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

data_loader_7_train = DataLoader(dataset_7, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
data_loader_7_test = DataLoader(dataset_7_test, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

data_loader_14_train = DataLoader(dataset_14, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
data_loader_14_test = DataLoader(dataset_14_test, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

data_loader_all_train = DataLoader(dataset_all, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
data_loader_all_test = DataLoader(dataset_all_test, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [92]:
class ModelGenerator():
    """
    Because there are dropout layers in the networks, each training session will have inherent stochasticity. We 
    will build many models and pick the best one
    """
    def __init__(self, data_loader_train, data_loader_test):
        self.data_loader_train = data_loader_train
        self.data_loader_test = data_loader_test

    def train(self):
        """
        Model training function
    
        fa_module: FAM class object used for initial feature extraction from initial CLS and Feature tensors
        proj_module: Projection class object used to reduce FAM features for contrastive learning loss function
        supconloss_module: SupConLoss class object that calculates contrastive loss
        classifer: Classifier class object that outputs predictions from FAM features
        data_loader: DataLoader class object that splits data into batches for model training
        optimizer: optimizer object which helps models with parameters converge to proper feature weights
        classifier_loss_fn: loss function which helps the classifer converge to proper feature weights
    
        Returns:
            Tuple of average loss at [0] and accuracy at [1] for each epoch
        """
        # Set all networks to train mode
        self.fam.train()
        self.proj.train()
        self.supcon_loss.train()
        self.classifier.train()
    
        # Declare variables to measure batch performance
        correct = 0
        total_targets = 0
        n_batches = 0
        train_loss = 0
    
        for data in self.data_loader_train:
            # Start model training
            n_batches += 1
            self.optimizer.zero_grad()
            embs = data[0].squeeze(1)
            targets = data[1]
            """
            Model flow: FAM -> projection head -> supcon loss. Predictions are made from FAM features, while
            Supconloss is calculated from the projection head.
            """
            fam_output = self.fam(embs)   
            proj_output = self.proj(fam_output)
            supcon_loss = self.supcon_loss(proj_output, targets)
            classifier_output = self.classifier(fam_output)  
            classifier_loss = self.classifier_loss(classifier_output, targets)
    
            # Use a combined supconloss and classifer loss
            loss = supcon_loss + classifier_loss 
            loss.backward()  
            self.optimizer.step()  
            train_loss += loss.item()
    
            # Calculate accuracy for the batch
            preds = classifier_output.argmax(1)
            correct += np.sum(np.array(preds) == np.array(targets))
            total_targets += len(targets)
    
        # Calculate overall accuracy and loss
        accuracy = correct / total_targets
        average_loss = train_loss / n_batches
    
        return average_loss, accuracy

    def evaluate(self):
        """
        fa_module: optimized FAM object used to generate features
        classifier: optimized Classifier object used to classify features from fa_module
        data_loader: DataLoader class object that splits data into batches for model training
    
        Returns:
            accuracy of evaluation
        """
        # Set networks to eval mode
        self.fam.eval()  
        self.classifier.eval()
    
        correct = 0
        total = 0
    
        # Evaluation the FAM and Classifier
        with torch.no_grad():  
            for data in self.data_loader_test:
                embs = data[0].squeeze(1)  
                targets = data[1].tolist()
                
                fam_output = self.fam(embs)
                final_output = self.classifier(fam_output)
                preds = final_output.argmax(1).tolist()
                
                total += len(preds) 
                correct += np.sum(np.array(preds) == np.array(targets))  
    
        accuracy = correct / total
        return accuracy

    def gen_model(self):
        """
        Generates the best model based on test set accuracy
        """
        # Declare constants for networks
        HIDDEN_SIZE_1 = 256
        HIDDEN_SIZE_2 = 128
        DROPOUT_PERCENT = 0.3
        NUM_CLASSES = 3
        LEARNING_RATE = 0.001
        STEP_SIZE = 20
        GAMMA = 0.5
        STOP_EARLY_NUM = 10
        NUM_MODELS = 200

        # Keep track of the best test accuracy
        best_test_acc = 0
        for i in tqdm(range(NUM_MODELS)):
            self.fam = FAM(NUM_FEATURES, HIDDEN_SIZE_1, DROPOUT_PERCENT)
            self.proj = Projection(HIDDEN_SIZE_1, HIDDEN_SIZE_2)
            self.classifier = Classifier(HIDDEN_SIZE_1, NUM_CLASSES, DROPOUT_PERCENT)
            self.optimizer = optimizer = torch.optim.Adam(list(self.fam.parameters()) + 
                                             list(self.proj.parameters()) + 
                                             list(self.classifier.parameters()), 
                                            lr=LEARNING_RATE)
            self.scheduler = scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=STEP_SIZE, gamma=GAMMA)
            self.classifier_loss = nn.CrossEntropyLoss()
            self.supcon_loss = SupConLoss()
            i = 0
            best_acc = 0
            # Utilize 'stopping early' when 10 successive models have failed to improve
            while i < STOP_EARLY_NUM:
                loss, acc = self.train()  
                if acc > best_acc:
                    best_acc = acc
                    best_fam = self.fam.state_dict()  
                    best_proj = self.proj.state_dict() 
                    best_classifier = self.classifier.state_dict()
                    i = 0
                else:
                    i += 1
                scheduler.step()

            # Load the best training model for test set evaluation
            self.fam.load_state_dict(best_fam)
            self.proj.load_state_dict(best_proj)
            self.classifier.load_state_dict(best_classifier)
            test_accuracy = self.evaluate()
            
            if test_accuracy > best_test_acc:
                # If this model performs better than previous models, update overall best
                best_test_acc = test_accuracy
                self.overall_best_fam = best_fam
                self.overall_best_proj = best_proj
                self.overall_best_classifier = best_classifier
                print('Train Set Accuracy: ' + str(best_acc*100) + '%')
                print('Test Set Accuracy: ' + str(test_accuracy*100) + '%')
                
    def save_model(self, filepath):
        # Save the best models to disk
        torch.save({'fam_state_dict': self.overall_best_fam,
            'proj_state_dict': self.overall_best_proj,
            'classifier_state_dict': self.overall_best_classifier,
        }, filepath)


In [70]:
temp_0_generator = ModelGenerator(data_loader_0_train, data_loader_0_test)
temp_0_generator.gen_model()
temp_0_generator.save_model('temp_0_models.pth')

  0%|▍                                                                                 | 1/200 [00:08<27:23,  8.26s/it]

Train Set Accuracy: 53.029411764705884%
Test Set Accuracy: 55.125%


  1%|▊                                                                                 | 2/200 [00:19<33:20, 10.10s/it]

Train Set Accuracy: 52.38235294117647%
Test Set Accuracy: 55.875%


  2%|█▏                                                                                | 3/200 [00:46<57:51, 17.62s/it]

Train Set Accuracy: 54.470588235294116%
Test Set Accuracy: 58.25%


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [47:03<00:00, 14.12s/it]


In [71]:
temp_7_generator = ModelGenerator(data_loader_7_train, data_loader_7_test)
temp_7_generator.gen_model()
temp_7_generator.save_model('temp_7_models.pth')

  0%|▍                                                                                 | 1/200 [00:16<53:28, 16.12s/it]

Train Set Accuracy: 52.485714285714295%
Test Set Accuracy: 50.875%


  2%|██                                                                                | 5/200 [00:54<33:49, 10.41s/it]

Train Set Accuracy: 52.142857142857146%
Test Set Accuracy: 53.5%


  6%|████▊                                                                            | 12/200 [02:46<49:29, 15.79s/it]

Train Set Accuracy: 52.51428571428571%
Test Set Accuracy: 54.50000000000001%


 54%|██████████████████████████████████████████▊                                     | 107/200 [27:03<30:40, 19.79s/it]

Train Set Accuracy: 52.0%
Test Set Accuracy: 54.625%


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [51:27<00:00, 15.44s/it]


In [72]:
temp_14_generator = ModelGenerator(data_loader_14_train, data_loader_14_test)
temp_14_generator.gen_model()
temp_14_generator.save_model('temp_14_models.pth')

  0%|▍                                                                               | 1/200 [00:21<1:12:23, 21.83s/it]

Train Set Accuracy: 52.142857142857146%
Test Set Accuracy: 51.87500000000001%


  1%|▊                                                                                 | 2/200 [00:35<55:53, 16.94s/it]

Train Set Accuracy: 51.42857142857142%
Test Set Accuracy: 52.37500000000001%


  2%|██                                                                                | 5/200 [01:28<55:19, 17.02s/it]

...

  4%|██▊                                                                               | 7/200 [02:00<52:50, 16.43s/it]

Train Set Accuracy: 51.97142857142857%
Test Set Accuracy: 53.5%


 78%|██████████████████████████████████████████████████████████████▍                 | 156/200 [34:40<10:08, 13.82s/it]

Train Set Accuracy: 51.51428571428571%
Test Set Accuracy: 53.87499999999999%


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [42:42<00:00, 12.81s/it]


In [94]:
temp_all_generator = ModelGenerator(data_loader_all_train, data_loader_all_test)
temp_all_generator.gen_model()
temp_all_generator.save_model('temp_all_models.pth')

  0%|▍                                                                               | 1/200 [00:19<1:04:59, 19.59s/it]

Train Set Accuracy: 49.27619047619047%
Test Set Accuracy: 52.07692307692307%


  1%|▊                                                                               | 2/200 [00:44<1:14:14, 22.50s/it]

Train Set Accuracy: 50.114285714285714%
Test Set Accuracy: 52.730769230769226%


  2%|█▏                                                                              | 3/200 [01:11<1:21:54, 24.95s/it]

...

 16%|█████████████                                                                  | 33/200 [15:42<1:29:46, 32.26s/it]

Train Set Accuracy: 50.08571428571429%
Test Set Accuracy: 54.34615384615385%


 67%|████████████████████████████████████████████████████▎                         | 134/200 [1:04:06<33:58, 30.88s/it]

Train Set Accuracy: 50.32380952380953%
Test Set Accuracy: 54.46153846153846%


100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:33:57<00:00, 28.19s/it]
